# Exporting and Converting Spatial Data

## Introduction

## Learning Objectives

## Sample Datasets

## Installation and Setup

### Library Import and Initial Setup

## Installing and Loading Extensions

## Loading Sample Data

## Exporting to Pandas DataFrames

### Basic DataFrame Export

### Exporting Filtered Query Results

### Working with Geometry Columns in DataFrames

### Converting Geometries to Text for DataFrames

### Integrating with GeoPandas for Spatial Operations

### Performance Considerations for DataFrame Export

## Exporting to CSV Files

### Basic CSV Export

### Streaming Query Results to CSV

### Handling Geometries in CSV Exports

### When to Use CSV Export

## Exporting to JSON Files

### Basic JSON Export

### Exporting Filtered Query Results to JSON

### Handling Geometries in JSON Exports

### When to Use JSON Export

## Exporting to Excel Files

### Install the excel extension

### Installing and Loading the Excel Extension

### Basic Excel Export Without Geometries

### Including Spatial Information as Text Columns

### Exporting Filtered and Aggregated Results to Excel

### When to Use Excel Export

## Exporting to Parquet Files

### Key Advantages of Parquet

### Basic Parquet Export

### Exporting Filtered Query Results to Parquet

### Partitioned Parquet Writes

## Exporting to GeoJSON Format

### GeoJSON Structure and GDAL Integration

### Basic GeoJSON Export

### Exporting Filtered Subsets to GeoJSON

### Customizing GeoJSON Export Options

### When to Use GeoJSON

### Limitations and When to Avoid GeoJSON

## Exporting to Shapefile Format

### Shapefile Limitations and Constraints

### Why GeoPackage is Preferred

### Shapefile Export Syntax

## Exporting to GeoPackage Format

### Advantages of GeoPackage Over Legacy Formats

### Basic GeoPackage Export

### Exporting Filtered Subsets to GeoPackage

### When to Use GeoPackage Export

## Key Takeaways

## Exercises

### Exercise 1: Basic CSV Export with Geometry Handling

### Exercise 2: Filtered Exports to Multiple Formats

### Exercise 3: DataFrame Export and Manipulation

### Exercise 4: Exporting with Spatial Aggregations

### Exercise 5: GeoJSON Export for Web Mapping

### Exercise 6: GeoPackage Export for GIS Interoperability

### Exercise 7: Partitioned Parquet Export

### Exercise 8: Excel Export with Readable Geometries

### Exercise 9: Round-Trip Export and Import

### Exercise 10: Comprehensive Export Pipeline Challenge